# Özel Veri Kaynağı Oluşturma

**PyBroker**, [Yahoo Finance](https://www.pybroker.com/en/latest/reference/pybroker.data.html#pybroker.data.YFinance), [Alpaca](https://www.pybroker.com/en/latest/reference/pybroker.data .html#pybroker.data.Alpaca) ve herhangi bir ek kurulum gerektirmeden hemen kullanabileceğiniz [AKShare](https://github.com/akfamily/akshare). Ancak özel bir ihtiyacınız varsa veya farklı bir veri kaynağı kullanmak istiyorsanız **PyBroker** ayrıca kendi ```DataSource``` sınıfınızı oluşturmanıza da olanak tanır.


## DataSource'u Genişletme

Aşağıda verilen örnek kodda, CSV dosyasından veri yükleyen ```CSVDataSource``` adlı yeni bir ```DataSource``` uygulanmıştır. ```CSVDataSource```, ```prices.csv``` adlı dosyayı [Pandas DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) içine okur ve ardından sağlanan giriş parametrelerine göre bu DataFrame'deki verileri döndürür:

In [1]:
import pandas as pd
import pybroker
from pybroker.data import DataSource

class CSVDataSource(DataSource):
    
    def __init__(self):
        super().__init__()
        # Özel sütunları CSV'ye kaydedin.
        pybroker.register_columns('rsi')
    
    def _fetch_data(self, symbols, start_date, end_date, _timeframe, _adjust):
        df = pd.read_csv('data/prices.csv')
        df = df[df['symbol'].isin(symbols)]
        df['date'] = pd.to_datetime(df['date'])
        return df[(df['date'] >= start_date) & (df['date'] <= end_date)]

CSV dosyasındaki özel ```'rsi'``` sütununu **PyBroker** için kullanılabilir hale getirmek için, onu [pybroker.register_columns](https://www.pybroker.com/en/latest/) kullanarak kaydederiz. reference/pybroker.scope.html#pybroker.scope.register_columns). Bu, **PyBroker**'ın verileri işlerken bu özel sütunu kullanmasına olanak tanır.

Verileri özel DataSource'unuzdan döndürürken şu sütunları içermesi gerektiğini unutmayın: ```symbol```, ```date```, ```open```, ```high```, ```düşük``` ve ```close```; bu sütunlar **PyBroker** tarafından bekleniyor.

Artık CSV verilerini bir ```CSVDataSource``` örneğinden sorgulayabiliriz:

In [2]:
csv_data_source = CSVDataSource()
df = csv_data_source.query(['MCD', 'NKE', 'DIS'], '6/1/2021', '12/1/2021')
df

Loading bar data...
Loaded bar data: 0:00:00 



date symbol        open        high         low       close  \
0   2021-06-01    DIS  180.179993  181.009995  178.740005  178.839996   
1   2021-06-01    MCD  235.979996  235.990005  232.740005  233.240005   
2   2021-06-01    NKE  137.850006  138.050003  134.210007  134.509995   
3   2021-06-02    DIS  179.039993  179.100006  176.929993  177.000000   
4   2021-06-02    MCD  233.970001  234.330002  232.809998  233.779999   
..         ...    ...         ...         ...         ...         ...   
382 2021-11-30    MCD  247.380005  247.899994  243.949997  244.600006   
383 2021-11-30    NKE  168.789993  171.550003  167.529999  169.240005   
384 2021-12-01    DIS  146.699997  148.369995  142.039993  142.149994   
385 2021-12-01    MCD  245.759995  250.899994  244.110001  244.179993   
386 2021-12-01    NKE  170.889999  173.369995  166.679993  166.699997   

           rsi  
0    46.321532  
1    46.522926  
2    53.308085  
3    42.635256  
4    48.051484  
..         ...  
382  40.461178  
383  51.505558  
384  16.677555  
385  39.853689  
386  46.704527  

[387 rows x 7 columns]

Bir arka testte ```CSVDataSource``` kullanmak için yeni bir [Strategy](https://www.pybroker.com/en/latest/reference/pybroker.strategy.html#pybroker.strategy.Strategy) nesnesi oluşturuyoruz ve özel ```DataSource``` değerini iletin:

In [3]:
from pybroker import Strategy

def buy_low_sell_high_rsi(ctx):
    pos = ctx.long_pos() 
    if not pos and ctx.rsi[-1] < 30:
        ctx.buy_shares = 100
    elif pos and ctx.rsi[-1] > 70:
        ctx.sell_shares = pos.shares

strategy = Strategy(csv_data_source, '6/1/2021', '12/1/2021')
strategy.add_execution(buy_low_sell_high_rsi, ['MCD', 'NKE', 'DIS'])
result = strategy.backtest()
result.orders

Backtesting: 2021-06-01 00:00:00 to 2021-12-01 00:00:00

Loading bar data...
Loaded bar data: 0:00:00 

Test split: 2021-06-01 00:00:00 to 2021-12-01 00:00:00


  0% (0 of 129) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--
100% (129 of 129) |######################| Elapsed Time: 0:00:00 Time:  0:00:00



Finished backtest: 0:00:02


type symbol       date  shares  limit_price  fill_price  fees
id                                                               
1    buy    NKE 2021-09-21     100          NaN      154.86   0.0
2   sell    NKE 2021-11-04     100          NaN      173.82   0.0
3    buy    DIS 2021-11-16     100          NaN      159.40   0.0

Özel ```rsi``` sütununu **PyBroker** ile kaydettiğimiz için bu sütuna [ExecContext](https://www.pybroker.com/en/latest/reference/pybroker.php) adresinden erişilebileceğini unutmayın. context.html#pybroker.context.ExecContext) ```ctx.rsi``` kullanarak.

## Pandas DataFrame'i Kullanma

Kendi [DataSource'unuzu](https://www.pybroker.com/en/latest/reference/pybroker.data.html#pybroker.data.DataSource) uygulama esnekliğine ihtiyacınız yoksa, bir [Bunun yerine Pandas DataFrame'i](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) bir ```Strateji```'ye dönüştürün.

Göstermek için önceki örnek şu şekilde yeniden uygulanabilir:

In [4]:
df = pd.read_csv('data/prices.csv')
df['date'] = pd.to_datetime(df['date'])

pybroker.register_columns('rsi')

strategy = Strategy(df, '6/1/2021', '12/1/2021')
strategy.add_execution(buy_low_sell_high_rsi, ['MCD', 'NKE', 'DIS'])
result = strategy.backtest()
result.orders

Backtesting: 2021-06-01 00:00:00 to 2021-12-01 00:00:00

Test split: 2021-06-01 00:00:00 to 2021-12-01 00:00:00


  0% (0 of 129) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--
100% (129 of 129) |######################| Elapsed Time: 0:00:00 Time:  0:00:00



Finished backtest: 0:00:00


type symbol       date  shares  limit_price  fill_price  fees
id                                                               
1    buy    NKE 2021-09-21     100          NaN      154.86   0.0
2   sell    NKE 2021-11-04     100          NaN      173.82   0.0
3    buy    DIS 2021-11-16     100          NaN      159.40   0.0